# --- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?


## Approach
* Treat the data as a grid.
* (x,y) being horizontal position and vertical position.
* We check for a character that is not a period nor a number.
    * Digit characters can be ~~converted into 1~~ left alone,
    * Period characters can be converted into 0,
    * Symbol characters can be converted into A
* Positions are always equals or greater than 0 but less than the maximum number of positions.
* We check for valid positions that are -1, 0, or +1 around the coordinates for a symbol that is not a period or number.
* The checked position cannot be part of the original digits.

### Example 1
From the sample data, 467 does not have numbers above or to the left of it.
The digits occupy positions (0,0) (1,0) (2,0).
* (3,0) right 
* (0,1) below
* (1,1) below
* (2,1) below
* (3,1) diagonal - where a symbol exists.

In [100]:
import re

In [101]:
sampleData = '''467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..'''

In [102]:
heightOfGrid = 0
widthOfGrid = 0
for line in sampleData.splitlines():
    # line = re.sub(r'\d', "1", line)
    line = re.sub(r'\.', "0", line)
    line = re.sub(r'\D', "A", line)
    heightOfGrid += 1 
    widthOfGrid = len(line)
    print(line)

print(f'The size of the matrix is {widthOfGrid} by {heightOfGrid}')

4670011400
000A000000
0035006330
000000A000
617A000000
00000A0580
0059200000
0000007550
000A0A0000
0664059800
The size of the matrix is 10 by 10


In [103]:
count = 0
line = 0

# Index where the digits start/end
for digits in sampleData.splitlines():
    line += 1
    for match in re.finditer(r'\d+', digits):
        count += 1
        print("match", count, "on line", line, "of length", len(match.group()), "start index", match.start(), "End index", match.end())

match 1 on line 1 of length 3 start index 0 End index 3
match 2 on line 1 of length 3 start index 5 End index 8
match 3 on line 3 of length 2 start index 2 End index 4
match 4 on line 3 of length 3 start index 6 End index 9
match 5 on line 5 of length 3 start index 0 End index 3
match 6 on line 6 of length 2 start index 7 End index 9
match 7 on line 7 of length 3 start index 2 End index 5
match 8 on line 8 of length 3 start index 6 End index 9
match 9 on line 10 of length 3 start index 1 End index 4
match 10 on line 10 of length 3 start index 5 End index 8


In [104]:
count = 0
line = 0

# Index where the symbols are
for symbols in sampleData.splitlines():
    line += 1
    symbols = re.sub(r'\.', "0", symbols)
    symbols = re.sub(r'\D', "A", symbols)
    for match in re.finditer(r'A', symbols):
        count += 1
        print("match", count, "on line", line, match.group(), "start index", match.start(), "End index", match.end())

match 1 on line 2 A start index 3 End index 4
match 2 on line 4 A start index 6 End index 7
match 3 on line 5 A start index 3 End index 4
match 4 on line 6 A start index 5 End index 6
match 5 on line 9 A start index 3 End index 4
match 6 on line 9 A start index 5 End index 6


From the above we have the indicies where the digits begin/end and where the symbols are.

Where the digits start, we look at positions N, NW, W, SW, and W.<br>
Where the digits End, we look at positions N, NE, E, SE, and E.<br>
For 3+ digits, in the middle, we look at positions N and S.

We must do a check if the digits are at the edges of the matrix.

Information needed:
* Digits
    * Start/end index
    * Line
    * Length
* Position on Symbols
    * Index
    * Line

Store the information into a dictionary:
```python
# For the digit dictionary
digitDictionary = {
    count   :{
        "digits"        : match.group()
        "indexStart"    : match.start()
        # "indexEnd"    : match.end()
        "indexLine"     : line
        "indexLength"   : len(match.group()) 
    },
    "1"       :{
        "digits"        : 467
        "indexStart"    : 0
        # "indexEnd"    : 3
        "indexLine"     : 1
        "indexLength"   : 3
    },
    "2"       :{
        "digits"        : 114
        "indexStart"    : 5
        # "indexEnd"    : 8
        "indexLine"     : 1
        "indexLength"   : 3
    }
}

# For the symbol dictionary
synbolDictionary = {
    count : {
        "indexStart"    : match.start()
        "indexLine"     : line
    },

    "1"   : {
        "indexStart"    : 3
        "indexLine"     : 2
    },
    "2"   : {
        "indexStart"    : 7
        "indexLine"     : 4
    },
}
```


In [105]:
digitsDictionary = {}
count = 0
line = 0

for digits in sampleData.splitlines():
    for match in re.finditer(r'\d+', digits):
        tempDictionary = {}
        tempDictionary["digits"] = match.group()
        tempDictionary["indexStart"] = match.start()
        # tempDictionary["indexEnd"] = match.end()
        tempDictionary["indexLine"] = line
        tempDictionary["indexLength"] = len(match.group())
        digitsDictionary[f'{count}'] = tempDictionary
        count += 1
    line += 1

In [106]:
symbolDictionary = {}
count = 0
line = 0

# Index where the symbols are
for symbols in sampleData.splitlines():
    symbols = re.sub(r'\.', "0", symbols)
    symbols = re.sub(r'\D', "A", symbols)
    for match in re.finditer(r'A', symbols):
        tempDictionary = {}
        tempDictionary["indexStart"] = match.start()
        tempDictionary["indexLine"] = line
        symbolDictionary[f'{count}'] = tempDictionary
        count += 1
    line += 1

In [107]:
for keys in digitsDictionary:
    if digitsDictionary[keys]['indexLength'] == 3:
        print(f'The digits {digitsDictionary[keys]['digits']} occupies positions ({digitsDictionary[keys]['indexStart']},{digitsDictionary[keys]['indexLine']}), ({digitsDictionary[keys]['indexStart']+1},{digitsDictionary[keys]['indexLine']}), ({digitsDictionary[keys]['indexStart']+2},{digitsDictionary[keys]['indexLine']})')
    elif digitsDictionary[keys]['indexLength'] == 2:
        print(f'The digits {digitsDictionary[keys]['digits']} occupies positions ({digitsDictionary[keys]['indexStart']},{digitsDictionary[keys]['indexLine']}), ({digitsDictionary[keys]['indexStart']+1},{digitsDictionary[keys]['indexLine']})')

The digits 467 occupies positions (0,0), (1,0), (2,0)
The digits 114 occupies positions (5,0), (6,0), (7,0)
The digits 35 occupies positions (2,2), (3,2)
The digits 633 occupies positions (6,2), (7,2), (8,2)
The digits 617 occupies positions (0,4), (1,4), (2,4)
The digits 58 occupies positions (7,5), (8,5)
The digits 592 occupies positions (2,6), (3,6), (4,6)
The digits 755 occupies positions (6,7), (7,7), (8,7)
The digits 664 occupies positions (1,9), (2,9), (3,9)
The digits 598 occupies positions (5,9), (6,9), (7,9)


In [108]:
for symbol in symbolDictionary:
    print(f'A symbol occupies position ({symbolDictionary[symbol]['indexStart']},{symbolDictionary[symbol]['indexLine']})')

A symbol occupies position (3,1)
A symbol occupies position (6,3)
A symbol occupies position (3,4)
A symbol occupies position (5,5)
A symbol occupies position (3,8)
A symbol occupies position (5,8)


We can now iterate through the `digitsDictionary` to see if sees with any locations in the `symbolsDictionary`.<br>
We will continue usign zero-based numbering.

Does a symbol exist in a position that is:
```
(-1,-1)  ( 0,-1)  (+1,-1)

(-1, 0)           (+1, 0)

(-1,+1)  ( 0,+1)  (+1,+1)
```
From our digits' positions.
If a symbol is found in the radius, add digits and move to the next set of digits.

In [109]:
for keys in digitsDictionary:
    digitLength = 0
    while digitLength < digitsDictionary[keys]['indexLength']:
        print((digitsDictionary[keys]['indexStart'] + digitLength),digitsDictionary[keys]['indexLine'])
        digitLength += 1
            

0 0
1 0
2 0
5 0
6 0
7 0
2 2
3 2
6 2
7 2
8 2
0 4
1 4
2 4
7 5
8 5
2 6
3 6
4 6
6 7
7 7
8 7
1 9
2 9
3 9
5 9
6 9
7 9


In [110]:
horizontalRange = [-1, 0, 1]
verticalRange   = [-1, 0, 1]

for keys in digitsDictionary:
    digitLength = 0
    while digitLength < digitsDictionary[keys]['indexLength']:
        for i in horizontalRange:
            for j in verticalRange:
                print((digitsDictionary[keys]['indexStart'] + digitLength + j), (digitsDictionary[keys]['indexLine'] + i))
        digitLength += 1

-1 -1
0 -1
1 -1
-1 0
0 0
1 0
-1 1
0 1
1 1
0 -1
1 -1
2 -1
0 0
1 0
2 0
0 1
1 1
2 1
1 -1
2 -1
3 -1
1 0
2 0
3 0
1 1
2 1
3 1
4 -1
5 -1
6 -1
4 0
5 0
6 0
4 1
5 1
6 1
5 -1
6 -1
7 -1
5 0
6 0
7 0
5 1
6 1
7 1
6 -1
7 -1
8 -1
6 0
7 0
8 0
6 1
7 1
8 1
1 1
2 1
3 1
1 2
2 2
3 2
1 3
2 3
3 3
2 1
3 1
4 1
2 2
3 2
4 2
2 3
3 3
4 3
5 1
6 1
7 1
5 2
6 2
7 2
5 3
6 3
7 3
6 1
7 1
8 1
6 2
7 2
8 2
6 3
7 3
8 3
7 1
8 1
9 1
7 2
8 2
9 2
7 3
8 3
9 3
-1 3
0 3
1 3
-1 4
0 4
1 4
-1 5
0 5
1 5
0 3
1 3
2 3
0 4
1 4
2 4
0 5
1 5
2 5
1 3
2 3
3 3
1 4
2 4
3 4
1 5
2 5
3 5
6 4
7 4
8 4
6 5
7 5
8 5
6 6
7 6
8 6
7 4
8 4
9 4
7 5
8 5
9 5
7 6
8 6
9 6
1 5
2 5
3 5
1 6
2 6
3 6
1 7
2 7
3 7
2 5
3 5
4 5
2 6
3 6
4 6
2 7
3 7
4 7
3 5
4 5
5 5
3 6
4 6
5 6
3 7
4 7
5 7
5 6
6 6
7 6
5 7
6 7
7 7
5 8
6 8
7 8
6 6
7 6
8 6
6 7
7 7
8 7
6 8
7 8
8 8
7 6
8 6
9 6
7 7
8 7
9 7
7 8
8 8
9 8
0 8
1 8
2 8
0 9
1 9
2 9
0 10
1 10
2 10
1 8
2 8
3 8
1 9
2 9
3 9
1 10
2 10
3 10
2 8
3 8
4 8
2 9
3 9
4 9
2 10
3 10
4 10
4 8
5 8
6 8
4 9
5 9
6 9
4 10
5 10
6 10
5 8
6 8
7 8
5 9
6 9
7 9
5 10

In [111]:
horizontalRange = [-1, 0, 1]
verticalRange   = [-1, 0, 1]

sumOfParts = []

for digitKeys in digitsDictionary:
    for symbolKeys in symbolDictionary:
        digitLength = 0
        while digitLength < digitsDictionary[digitKeys]['indexLength']:
            for i in horizontalRange:
                for j in verticalRange:
                    if (digitsDictionary[digitKeys]['indexStart'] + digitLength + j) == symbolDictionary[symbolKeys]['indexStart'] and (digitsDictionary[digitKeys]['indexLine'] + i) == symbolDictionary[symbolKeys]['indexLine']:
                        print(f'''a symbol is found at {symbolDictionary[symbolKeys]['indexStart']},{symbolDictionary[symbolKeys]['indexLine']} for the digits {digitsDictionary[digitKeys]['digits']}''')
                        sumOfParts.append(digitsDictionary[digitKeys]['digits'])
            digitLength += 1

print(sumOfParts)

a symbol is found at 3,1 for the digits 467
a symbol is found at 3,1 for the digits 35
a symbol is found at 3,1 for the digits 35
a symbol is found at 6,3 for the digits 633
a symbol is found at 6,3 for the digits 633
a symbol is found at 3,4 for the digits 617
a symbol is found at 5,5 for the digits 592
a symbol is found at 5,8 for the digits 755
a symbol is found at 3,8 for the digits 664
a symbol is found at 3,8 for the digits 664
a symbol is found at 5,8 for the digits 598
a symbol is found at 5,8 for the digits 598
['467', '35', '35', '633', '633', '617', '592', '755', '664', '664', '598', '598']


In [112]:
horizontalRange = [-1, 0, 1]
verticalRange   = [-1, 0, 1]

sumOfParts = []

for digitKeys in digitsDictionary:
    for symbolKeys in symbolDictionary:
        digitLength = 0
        
        # while symbolFound == False:
        while digitLength < digitsDictionary[digitKeys]['indexLength']:
            for i in horizontalRange:
                for j in verticalRange:
                    if (digitsDictionary[digitKeys]['indexStart'] + digitLength + j) == symbolDictionary[symbolKeys]['indexStart'] and (digitsDictionary[digitKeys]['indexLine'] + i) == symbolDictionary[symbolKeys]['indexLine']:
                        print(f'''a symbol is found at {symbolDictionary[symbolKeys]['indexStart']},{symbolDictionary[symbolKeys]['indexLine']} for the digits {digitsDictionary[digitKeys]['digits']}''')
                        sumOfParts.append(int(digitsDictionary[digitKeys]['digits']))
                        symbolFound = True
            digitLength += 1
sumOfParts = list(dict.fromkeys(sumOfParts))
print(sum(sumOfParts))

a symbol is found at 3,1 for the digits 467
a symbol is found at 3,1 for the digits 35
a symbol is found at 3,1 for the digits 35
a symbol is found at 6,3 for the digits 633
a symbol is found at 6,3 for the digits 633
a symbol is found at 3,4 for the digits 617
a symbol is found at 5,5 for the digits 592
a symbol is found at 5,8 for the digits 755
a symbol is found at 3,8 for the digits 664
a symbol is found at 3,8 for the digits 664
a symbol is found at 5,8 for the digits 598
a symbol is found at 5,8 for the digits 598
4361


With the above code, I would have liked to added the necessary code to when a symbol was found in the radial of a digit, to skip searching for any more symbols and move onto the next set of digits.

In [113]:
with open("EngineSchematic.txt") as f:
    data = f.read()

In [114]:
digitsDictionary = {}
count = 0
line = 0

for digits in data.splitlines():
    for match in re.finditer(r'\d+', digits):
        tempDictionary = {}
        tempDictionary["digits"] = match.group()
        tempDictionary["indexStart"] = match.start()
        # tempDictionary["indexEnd"] = match.end()
        tempDictionary["indexLine"] = line
        tempDictionary["indexLength"] = len(match.group())
        digitsDictionary[f'{count}'] = tempDictionary
        count += 1
    line += 1

In [115]:
symbolDictionary = {}
count = 0
line = 0

# Index where the symbols are
for symbols in data.splitlines():
    symbols = re.sub(r'\.', "0", symbols)
    symbols = re.sub(r'\D', "A", symbols)
    for match in re.finditer(r'A', symbols):
        tempDictionary = {}
        tempDictionary["indexStart"] = match.start()
        tempDictionary["indexLine"] = line
        symbolDictionary[f'{count}'] = tempDictionary
        count += 1
    line += 1

In [125]:
horizontalRange = [-1, 0, 1]
verticalRange   = [-1, 0, 1]

sumOfParts = []

for digitKeys in digitsDictionary:
    for symbolKeys in symbolDictionary:
        digitLength = 0
        while digitLength < digitsDictionary[digitKeys]['indexLength']:
            for i in horizontalRange:
                for j in verticalRange:
                    if (digitsDictionary[digitKeys]['indexStart'] + digitLength + j) == symbolDictionary[symbolKeys]['indexStart'] and (digitsDictionary[digitKeys]['indexLine'] + i) == symbolDictionary[symbolKeys]['indexLine']:
                        # print(f'''a symbol is found at {symbolDictionary[symbolKeys]['indexStart']},{symbolDictionary[symbolKeys]['indexLine']} for key {digitsDictionary[digitKeys]} digits {digitsDictionary[digitKeys]['digits']}''')
                        sumOfParts.append(digitsDictionary[digitKeys])
                        # [i for n, i in enumerate(sumOfParts) if i not in sumOfParts[n + 1:]]
            digitLength += 1

In [140]:
# Remove duplicate entries.
sumOfParts = [i for n, i in enumerate(sumOfParts) if i not in sumOfParts[n + 1:]]

In [151]:
sumTotal = []
for line in sumOfParts:
    sumTotal.append(int(line['digits']))

sum(sumTotal)

539713